For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

$$\begin{aligned}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
\end{aligned}$$

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\text{IRR}(K)} \\
      h'(K) &= \frac{\text{IRR}(K)g'(K) - g(K)\text{IRR}'(K)}{\text{IRR}(K)^2} \\
      h''(K) &= \frac{\text{IRR}(K)g''(K)-\text{IRR}''(K)g(K) -2\cdot\text{IRR}'(K)g'(K)}{\text{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\text{IRR}'(K)^2g(K)}{\text{IRR}(K)^3}.
    \end{split}
  \end{equation*}
  
For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

Let $m$ denote the payment frequency ($m=2$ for semi-annual payment frequency), and let $N = T_N-T_n$ denote the tenor of the swap (number of years), the partial derivatives on the IRR function $\text{IRR}(S)$ given by:
\begin{equation*}
\begin{split}
\text{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\text{IRR}'(K)&=-\frac{1}{K}\text{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\text{IRR}''(K)&=-\frac{2}{K}\text{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

These results will need to be generalised to handle the case for $m=2$ to be consistent with the semi-annual payment frequency swap market data provided.

---
For CMS rate payment, since $g(F)=F$, we have the derivatives:

\begin{equation*}
\begin{split}
g(K) &= K \\
g'(K) &= 1 \\
g''(K) &= 0
\end{split}
\end{equation*}

# Imports and data wrangling

OIS is in semiannual df. need to also include for quarterly period

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq
from scipy.integrate import quad
from scipy.stats import norm
from scipy.interpolate import interp1d

ois = pd.read_csv('./data/ois.csv')     # load discounting curve
fsr = pd.read_csv('./data/sabr_params.csv')     # load discounting curve
irs = pd.read_csv('./data/irs.csv')             # load par swap rates
# get discount factors up to quarterly periods
# new_expiry = np.arange(0.25, ois.years.max()+.25, .25)
# ois = ois.set_index("years").reindex(new_expiry)
# ois.f = ois.f.bfill()
# ois.reset_index(inplace=True)
# ois["df"] = 1 / (1 + ois["f"] / 360) ** (90)  # get quarterly DF
# ois["df"] = ois["df"].cumprod()               # cum prod to get all DF

In [2]:
new_expiry = np.arange(0.25, ois["years"].max() + 0.25, 0.25)

# Expand the dataframe without affecting other columns
ois = ois.set_index("years").reindex(new_expiry)

# Fill forward/backward only selected columns
ois["f"] = ois["f"].bfill()  # Fill missing forward rates
ois = ois.ffill()  # Fill other missing values from above

# Reset index
ois.reset_index(inplace=True)
ois.rename(columns={"index": "years"}, inplace=True)

# Compute quarterly discount factors
ois["df"] = ois.apply(lambda row: 1 / ((1 + row['f'] / 360) ** (90)), axis=1)
ois["df_cumprod"] = ois["df"].cumprod()
ois.tail(10)

,years,Tenor,Product,Rate,f,cumsum_df,df,df_cumprod
110,27.75,20y,OIS,0.00525,0.006028,26.182081,0.998494,0.858997
111,28.00,20y,OIS,0.00525,0.006028,27.039784,0.998494,0.857703
112,28.25,20y,OIS,0.00525,0.006028,27.039784,0.998494,0.856412
113,28.50,20y,OIS,0.00525,0.006028,27.039784,0.998494,0.855122
114,28.75,20y,OIS,0.00525,0.006028,27.039784,0.998494,0.853834
115,29.00,20y,OIS,0.00525,0.006028,27.892332,0.998494,0.852548
116,29.25,20y,OIS,0.00525,0.006028,27.892332,0.998494,0.851264
117,29.50,20y,OIS,0.00525,0.006028,27.892332,0.998494,0.849983
118,29.75,20y,OIS,0.00525,0.006028,27.892332,0.998494,0.848703
119,30.00,30y,OIS,0.00550,0.006028,28.739756,0.998494,0.847424


In [3]:
1/((1+0.002498/360)**(.5*360)*(1+0.003493/360)**(.5*360)*(1+0.003495/360)**(1*360)*(1+0.003545/360)**(1*360))

0.9900147343578931

In [4]:
## IRR funcions
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative
# Payoff functions
def g_0(K):
    return K

def g_1(K):
    return 1.0

def g_2(K):
    return 0.0

def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative
    
def black76_pay(F, K, expiry, tenor, sigma):
    """
    Return value of payer swaption via Black Normal model only
    parameters
        F: forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * (sigma**2) * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = F * norm.cdf(d_1) - K * norm.cdf(d_2)
    return black_option
    
def black76_rec(F, K, expiry, tenor, sigma):
    """
    Return value of receiver swaption via Black Normal model Only
    parameters
        F: Forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * (sigma**2) * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = K * norm.cdf(-d_2) - F * norm.cdf(-d_1)
    return black_option

def SABR(F, K, T, alpha, rho, nu, beta=0.9):
    """
    Use SABR model to return sigma, set beta to be 0.9
    """
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [5]:
def rec_integrand(F, K, expiry, tenor, alpha, rho, nu, m):
    """
    Return value of receiver integrand per CMS rate payoff. 
    formula: 2nd derivative h * V_rec(per SABR valuation) 
    parameters:
        m: payment freq per year, e.g. 2 is semi-annual
        F: par swap rate
        K: strike of CMS
        expiry: years to expiry
        alpha: SABR param
        rho: SABR param
        nu: SABR param
    """
    sigma_sabr = SABR(F, K, expiry, alpha, rho, nu, beta=0.9)
    sabr_pricing = black76_rec(F, K, expiry, tenor, sigma_sabr)
    return h_2(K, m, tenor) * sabr_pricing

def pay_integrand(F, K, expiry, tenor, alpha, rho, nu, m):
    """
    Return value of receiver integrand per CMS rate payoff. 
    formula: 2nd derivative h * V_rec(per SABR valuation) 
    parameters:
        m: payment freq per year, e.g. 2 is semi-annual
        F: par swap rate
        K: strike of CMS
        expiry: years to expiry
        alpha: SABR param
        rho: SABR param
        nu: SABR param
    """
    sigma_sabr = SABR(F, K, expiry, alpha, rho, nu, beta=0.9)
    sabr_pricing = black76_pay(F, K, expiry, tenor, sigma_sabr)
    return h_2(K, m, tenor) * sabr_pricing

def v_0(F, expiry, tenor, alpha, rho, nu, m):
    """
    Return CMS pay off based of static replication discounted to today's price. Uses SABR model
    
    """
    df_expiry = ois.loc[ois.years == expiry]['df'].values[0]
    int_1 = IRR_0(F, m, tenor) * quad(lambda x: rec_integrand(F, x, expiry, tenor, alpha, rho, nu, m), 0.0, F)[0]
    int_2 = IRR_0(F, m, tenor) * quad(lambda x: pay_integrand(F, x, expiry, tenor, alpha, rho, nu, m), F, 0.1)[0]

    return df_expiry * F + int_1 + int_2



In [43]:
def forward_cms_coupon(maturity, tenor, m):
    """
    Return PV of receiver leg for CMS as a series of periodic payments. 
    parameters:
        maturity: CMS maturity or swap tenor 
        tenor: tenor of CMS payments to be made
        m: frequency of payments per year
        par_swap_0: first swap rate at t=0 to be paid out in next payment date
    """
    # Data preparation
    df_fwd_irs = fsr.loc[fsr.Tenor == maturity].copy()
    new_expiry = np.arange(0, tenor, 1/m)
    ls_v0 = []
    df_tmp = pd.DataFrame({"Expiry": new_expiry, 
                            "Tenor" : maturity})
    
    # interpolate sabr params
    for col in ["alpha", "rho", "nu"]:
        interp_func = interp1d(df_fwd_irs["Expiry"], 
                            df_fwd_irs[col], 
                            kind="linear", 
                            fill_value="extrapolate")
        df_tmp[col] = interp_func(new_expiry)
    # add par IRS  then interpolate par swap rates
    par_swap_0 = irs.loc[irs.years==maturity,'Rate'].values[0]        # from part 1
    df_fwd_irs.loc[0] = {"Expiry": 0, "f_irs": par_swap_0} 
    df_fwd_irs.sort_values(by="Expiry", inplace=True)
    interp_func = interp1d(df_fwd_irs["Expiry"], 
                        df_fwd_irs['f_irs'], 
                        kind="linear", 
                        fill_value="extrapolate")
    df_tmp['f_irs'] = interp_func(new_expiry)

    # iterate through fixing dates
    for expiry in new_expiry:
        if expiry == 0:
            coupon = df_tmp.iloc[0]['f_irs']
            print(f"{expiry}: {coupon}")
            ls_v0.append(coupon)       # first payment is fixed at par swap for first period
            continue

        F = df_tmp.loc[df_tmp.Expiry == expiry].f_irs.values[0]
        alpha = df_tmp.loc[df_tmp.Expiry == expiry].alpha.values[0]
        rho = df_tmp.loc[df_tmp.Expiry == expiry].rho.values[0]
        nu = df_tmp.loc[df_tmp.Expiry == expiry].nu.values[0]

        coupon =  v_0(F, expiry, maturity, alpha, rho, nu, m)
        print(f"{expiry}: {coupon}")
        ls_v0.append(coupon)

    return np.sum(np.array(ls_v0)) / m, ls_v0




# CMS10y for 5 years

In [46]:
res, _ = forward_cms_coupon(10, 5, 2)
f"PV of a leg receiving CMS10y semi-annually over the next 5 years {res}"

0.0: 0.037
0.5: 0.03792091716886464
1.0: 0.03894330570756703
1.5: 0.03998114829044077
2.0: 0.04104293507586774
2.5: 0.042099057606126915
3.0: 0.04313440475205272
3.5: 0.04413699175965234
4.0: 0.04511331143154063
4.5: 0.04606039808285682


'PV of a leg receiving CMS10y semi-annually over the next 5 years 0.2077162349374848'

# CMS2y for 10 years

In [47]:
res, _ = forward_cms_coupon(2, 10, 4)
f"PV of a leg receiving CMS2y semi-annually over the next 10 years {res}"

0.0: 0.03
0.25: 0.030808845350039107
0.5: 0.031655733807368414
0.75: 0.032516123437848134
1.0: 0.033400314822721086
1.25: 0.03391425035352576
1.5: 0.034430771232710176
1.75: 0.03494660544612718
2.0: 0.03545963240710815
2.25: 0.03596806377458285
2.5: 0.03647195778947216
2.75: 0.03697042062669875
3.0: 0.037463247426572095
3.25: 0.03794670040021228
3.5: 0.038428261233161363
3.75: 0.03890445564398759
4.0: 0.03937554594996414
4.25: 0.03984135782453509
4.5: 0.04030321096273004
4.75: 0.040761003430498995
5.0: 0.04121513764556077
5.25: 0.041413105940389244
5.5: 0.04161992134384578
5.75: 0.041825505310786
6.0: 0.04202986011145039
6.25: 0.04223299138954071
6.5: 0.042434907494999036
6.75: 0.042635618949834614
7.0: 0.04283513801805505
7.25: 0.043026474023966574
7.5: 0.04322362470497321
7.75: 0.04341962707547243
8.0: 0.043614497475487214
8.25: 0.04380825278053032
8.5: 0.044000910273613664
8.75: 0.04419248753966747
9.0: 0.04438300237835703
9.25: 0.044572472732074174
9.5: 0.04476091662650051
9.75: 0.

'PV of a leg receiving CMS2y semi-annually over the next 10 years 0.3954398264641491'

# Compare swaps

In [52]:
df = fsr.pivot(index='Expiry',
                                columns='Tenor', 
                                values='f_irs')

In [55]:
ls_expiries = df.index
ls_maturities = df.columns
ls_maturities, ls_expiries

(Index([1, 2, 3, 5, 10], dtype='int64', name='Tenor'),
 Index([1, 5, 10], dtype='int64', name='Expiry'))

In [60]:
df_cms =  df.copy()
dict_cms = {}

for maturity in ls_maturities:
    _, dict_cms[maturity] = forward_cms_coupon(maturity, 10, 2)
    
    for expiry in ls_expiries:
        df_cms.at[expiry,maturity] = dict_cms[maturity][int(expiry*.5)]


0.0: 0.028
0.5: 0.029135797909989773
1.0: 0.030329806492162832
1.5: 0.03155056373517937
2.0: 0.03271193058021015
2.5: 0.03380633494035984
3.0: 0.03489220558850133
3.5: 0.03602336388081591
4.0: 0.037263719152298404
4.5: 0.03866387775923684
5.0: 0.04006492038312771
5.5: 0.04042462976395484
6.0: 0.04078939011631152
6.5: 0.04114925950239353
7.0: 0.041504232092363265
7.5: 0.04184746383973849
8.0: 0.04219274938275557
8.5: 0.042533358626084095
9.0: 0.042869420585835594
9.5: 0.04320108560452349
0.0: 0.03
0.5: 0.03166260077163833
1.0: 0.033421783116019295
1.5: 0.03446953929681047
2.0: 0.03551545125552202
2.5: 0.03654318269142972
3.0: 0.03754772432660822
3.5: 0.03852374618215161
4.0: 0.03947991382760993
4.5: 0.04041456454512342
5.0: 0.04133186734643996
5.5: 0.04174750070561378
6.0: 0.04216779351519466
6.5: 0.04258270405106817
7.0: 0.042992315624429234
7.5: 0.04338971261501381
8.0: 0.04378903789774094
8.5: 0.044183459332120804
9.0: 0.04457313064513074
9.5: 0.04495820936570108
0.0: 0.0315
0.5: 0.0

In [62]:
df_cms

Tenor,1,2,3,5,10
Expiry,,,,,
1,0.028000,0.030000,0.031500,0.033000,0.037000
5,0.030330,0.033422,0.034260,0.035526,0.038943
10,0.033806,0.036543,0.037231,0.038458,0.042099


In [63]:
df

Tenor,1,2,3,5,10
Expiry,,,,,
1,0.031922,0.033217,0.033982,0.035238,0.038419
5,0.039274,0.040075,0.040072,0.041093,0.043634
10,0.042190,0.043116,0.044097,0.046249,0.053458
